In [1]:
import os
os.environ["HTTPS_PROXY"] = ""
os.environ["HTTP_PROXY"] = ""

In [2]:
import time
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

import warnings
warnings.filterwarnings('ignore')

In [3]:
path = "Analytics/Customer Projects/T-Mobile-Proj 10869 (Custom Optics_Spectrum)/File from the Customer/LN_RETRO.csv.gz"

In [9]:
from dsgtools import azure
data = pd.read_adls(path, compression = "gzip", reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(data.shape)

(3522391, 141)


In [10]:
data.CRID.is_unique

True

## overall clean

In [12]:
data["address"] = np.where(data.ADDRESSLINE1.notnull(), np.where(data.ADDRESSLINE2.notnull(), data.ADDRESSLINE1 + " " + data.ADDRESSLINE2, data.ADDRESSLINE1), data.ADDRESSLINE1)
data[['FIRSTNAME', 'LASTNAME', 'BIRTHDATE', 'ADDRESSCITY', 'ADDRESSLINE1', 'ADDRESSLINE2', 'ADDRESSSTATE', 'ZIP_FINAL', 'PHONE', 'SSN', "address"]].isnull().sum()

FIRSTNAME            15
LASTNAME             43
BIRTHDATE             1
ADDRESSCITY           0
ADDRESSLINE1          3
ADDRESSLINE2    3310725
ADDRESSSTATE          0
ZIP_FINAL             0
PHONE                 0
SSN                   0
address               3
dtype: int64

In [13]:
Input_FirstName = "FIRSTNAME"
Input_LastName = "LASTNAME"
Input_StreetAddress = "address"
Input_City = "ADDRESSCITY"
Input_State = "ADDRESSSTATE"
Input_Zip = "ZIP_FINAL"
Input_SSN = "SSN"

data["sufficient_input"] = np.where((data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_Zip].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_StreetAddress].notnull() & data[Input_City].notnull() & data[Input_State].notnull()) | \
             (data[Input_FirstName].notnull() & data[Input_LastName].notnull() & data[Input_SSN].notnull()), 1, 0)
freq(data["sufficient_input"])

,Count,Pct,Cuml Count,Cuml Pct
sufficient_input,,,,
0,58.0,0.000016,58.0,0.000016
1,3522333.0,0.999984,3522391.0,1.000000


In [14]:
data = data[data["sufficient_input"] == 1]
from datetime import datetime
data["APPDATETIME_fmt"] = pd.to_datetime(data.APPDATETIME.str.slice(0,9))
data["APPDATETIME_fmt_F"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7)
data["APPDATETIME_fmt_F_dt"] = data['APPDATETIME_fmt'].astype(str).str.slice(0,4) + data['APPDATETIME_fmt'].astype(str).str.slice(5,7) + data['APPDATETIME_fmt'].astype(str).str.slice(8,10)
freq(data["APPDATETIME_fmt_F"] )

,Count,Pct,Cuml Count,Cuml Pct
APPDATETIME_fmt_F,,,,
202110,675577.0,0.191798,675577.0,0.191798
202111,738451.0,0.209648,1414028.0,0.401446
202112,767772.0,0.217973,2181800.0,0.619419
202201,665620.0,0.188971,2847420.0,0.808390
202202,674913.0,0.191610,3522333.0,1.000000


In [16]:
data["WO12M_IND_fmt"] = np.where(data.WO12M_IND.isnull(), -1, np.where(data.WO12M_IND == "1", 1, 0))
freq(data.WO12M_IND_fmt)

,Count,Pct,Cuml Count,Cuml Pct
WO12M_IND_fmt,,,,
-1,1490679.0,0.423208,1490679.0,0.423208
0,1642248.0,0.466239,3132927.0,0.889447
1,389406.0,0.110553,3522333.0,1.000000


In [17]:
data = data.sort_values(by = ['FIRSTNAME', 'LASTNAME', 'BIRTHDATE', 'ADDRESSCITY', 'ADDRESSSTATE', 'ZIP_FINAL', 'PHONE', 'SSN', "address", 
                              'APPDATETIME_fmt_F_dt', "WO12M_IND_fmt"], ascending = False)
print(data.shape)
data = data.drop_duplicates(subset = ['FIRSTNAME', 'LASTNAME', 'BIRTHDATE', 'ADDRESSCITY', 'ADDRESSSTATE', 'ZIP_FINAL', 'PHONE', 'SSN', "address", 'APPDATETIME_fmt_F_dt'], keep = "first")
print(data.shape)

(3522333, 147)
(3516579, 147)


In [18]:
3516579-3522333

-5754

In [22]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/tmobile_10869_full_file.parquet",
             overwrite = True)

In [3]:
from dsgtools import azure
data = pd.read_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/Full_sample_formatted_w_flags/ready_to_be_formatted/tmobile_10869_full_file.parquet")
print(data.shape)

(3516579, 149)


In [20]:
from datetime import datetime
data["BIRTHDATE_fmt"] = pd.to_datetime(data.BIRTHDATE.str.slice(0,10), errors = 'coerce', format = "%Y-%m-%d")
data["BIRTHDATE_fmt_dt"] = data['BIRTHDATE_fmt'].astype(str).str.slice(0,4) + data['BIRTHDATE_fmt'].astype(str).str.slice(5,7) + data['BIRTHDATE_fmt'].astype(str).str.slice(8,10)

## Sample for validation

In [4]:
freq(data.ACTIVATIONFLAG_NP15, data.ACTIVATIONFLAG_NP30, data.ACTIVATIONFLAG_NP60, data.ACTIVATIONFLAG_NP90, cross = False, observed = True)

,,,,Count,Pct,Cuml Count,Cuml Pct
ACTIVATIONFLAG_NP15,ACTIVATIONFLAG_NP30,ACTIVATIONFLAG_NP60,ACTIVATIONFLAG_NP90,,,,
0,0,0,0,1705296.0,4.849304e-01,1705296.0,0.484930
1,1,1,1,1811281.0,5.150690e-01,3516577.0,0.999999
Missing,Missing,Missing,Missing,2.0,5.687346e-07,3516579.0,1.000000


In [5]:
data = data[data.ACTIVATIONFLAG_NP15 == "1"]
print(data.shape)
freq(data.FRAUD_IND, data.FPD_NNI_IND, cross = False, observed = True)

(1811281, 149)


Count       Pct  Cuml Count  Cuml Pct
FRAUD_IND FPD_NNI_IND                                           
0         0            1668351.0  0.921089   1668351.0  0.921089
          1             126270.0  0.069713   1794621.0  0.990802
1         0              16658.0  0.009197   1811279.0  0.999999
Missing   Missing            2.0  0.000001   1811281.0  1.000000

In [6]:
data = data[(data.FRAUD_IND == "0") & ((data.FPD_NNI_IND == "0"))]
print(data.shape)
freq(data.WO6M_IND, data.WO9M_IND, data.WO12M_IND, data.WO_NNI_IND, cross = False, observed = True)

(1668351, 149)


Count       Pct  Cuml Count  \
WO6M_IND WO9M_IND WO12M_IND WO_NNI_IND                                    
0        0        0         0           1389815.0  0.833047   1389815.0   
                            1             51649.0  0.030958   1441464.0   
                  1         1             67914.0  0.040707   1509378.0   
         1        1         1             97151.0  0.058232   1606529.0   
1        1        1         1             61822.0  0.037056   1668351.0   

                                        Cuml Pct  
WO6M_IND WO9M_IND WO12M_IND WO_NNI_IND            
0        0        0         0           0.833047  
                            1           0.864005  
                  1         1           0.904712  
         1        1         1           0.962944  
1        1        1         1           1.000000

In [7]:
freq(data.WO12M_IND_fmt)

,Count,Pct,Cuml Count,Cuml Pct
WO12M_IND_fmt,,,,
0,1441464.0,0.864005,1441464.0,0.864005
1,226887.0,0.135995,1668351.0,1.000000


In [8]:
bivariate(data.APPDATETIME_fmt_F, data.WO12M_IND_fmt)

tag               WO12M_IND_fmt                                          \
stats                         N      PctN       Sum      Mean       WoE   
APPDATETIME_fmt_F                                                         
202110                 311227.0  0.186548   36839.0  0.118367 -0.159026   
202111                 345503.0  0.207093   42389.0  0.122688 -0.118259   
202112                 355786.0  0.213256   48046.0  0.135042 -0.008136   
202201                 321483.0  0.192695   47449.0  0.147594  0.095364   
202202                 334352.0  0.200409   52164.0  0.156015  0.160780   
Missing                     0.0  0.000000       0.0  0.000000  0.000000   
Total                 1668351.0  1.000000  226887.0  0.135995  0.000000   

tag                          
stats                    IV  
APPDATETIME_fmt_F            
202110             0.004451  
202111             0.002774  
202112             0.000014  
202201             0.001814  
202202             0.005490  
Missing            0.000000  
Total              0.014542

In [9]:
data.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/cleaned/tmobile_cleaned_11_20.parquet", overwrite = True)

### Add Weight

In [10]:
data["weight"] = 1
data_1 = data[(data.WO12M_IND_fmt == 1)].copy()
data_0 = data[(data.WO12M_IND_fmt == 0)].copy()
print(data_1.shape)
print(data_0.shape)

(226887, 150)
(1441464, 150)


In [11]:
data_0_sample = data_0.sample(n = 500_000 - data_1.shape[0], random_state=1)
data_0_sample['weight'] = data_0.shape[0]/(500_000 - data_1.shape[0])
print(data_0_sample['weight'].unique())
print(data_0_sample.shape)

[5.27790329]
(273113, 150)


In [12]:
data = pd.concat([data_1, data_0_sample], ignore_index= True)
print(data.shape)

(500000, 150)


In [13]:
data_sample = data.sample(n = 250_000, random_state=1)
freq(data_sample.WO12M_IND_fmt)

,Count,Pct,Cuml Count,Cuml Pct
WO12M_IND_fmt,,,,
0,136459.0,0.545836,136459.0,0.545836
1,113541.0,0.454164,250000.0,1.000000


In [14]:
data_sample.weight = 2* data_sample.weight

In [15]:
data_sample.to_adls("Analytics/RnD Projects/Product RnD/Credit/202311-CRD-Model-Revalidations/Formatted_Files/tmobile_250K_11_20.parquet", overwrite = True)